In [32]:
using Distributions
function simDat(nObs,nLoci,bMean,bStd,resStd)
    X  = [ones(nObs,1) rand(Binomial(2,0.5),(nObs,nLoci))]
    Xv = [ones(nObs,1) rand(Binomial(2,0.5),(nObs,nLoci))] # for validation
    b = rand(Normal(bMean,bStd),size(X,2))
    y = X*b + rand(Normal(0.0, resStd),nObs)
    return (y,X,Xv,b)
end

simDat (generic function with 1 method)

In [33]:
nObs     = 1000
nLoci    = 5
bMean    = 0.0
bStd     = 0.5
resStd   = 1.0
y,X,Xv,b = simDat(nObs,nLoci,bMean,bStd,resStd);

In [34]:
 trueb=b
 k=size(b,1)
 niter=10000  # This is the length of my chain
 b=zeros(k)   # This is a vector of plausible values
 s2e=resStd^2 # This is assumed known for now
 mcmcSamples=zeros(niter,k)
 for iter in 1:niter
    for thisParam in 1:k
        b[thisParam]=0.0              # Zero the parameter we are about to sample
       yadj=y-X*b                     # y is adjusted for everything but thisParam
        xcol=X[:,thisParam]           # This is the covariate column for thisParam
        bhat=(xcol'yadj/xcol'xcol)[1] # This is the LS solution conditioned on all the other parameters
        bhatvar=s2e/(xcol'xcol)[1]    # This is the variance of the LS solution  
        b[thisParam]=rand(Normal(bhat,sqrt(bhatvar)))  # Get a pluasible sample value for thisParam
    end
    mcmcSamples[iter,:]=b'            # Store the vector of samples for later inference
 end

In [35]:
mcmcmean=mean(mcmcSamples,1)';   # Note the need for the transpose
mcmcvarcov=cov(mcmcSamples);
lsmean=X'X\X'y;
lsvarcov=inv(X'X)*s2e;

In [36]:
[lsmean diag(lsvarcov) mcmcmean diag(mcmcvarcov)]  # Print out the results from MCMC and LS

6x4 Array{Float64,2}:
 -0.448407   0.0115167   -0.445319   0.0122095 
 -0.352886   0.0020387   -0.353681   0.00203636
 -0.219034   0.00203034  -0.219875   0.00208432
 -0.664648   0.00199424  -0.664473   0.00208737
  0.0831212  0.00209147   0.0828367  0.00216855
  0.569406   0.00194582   0.568271   0.0020361 

In [37]:
sampleEBV=X[:,2:end]*mcmcSamples[:,2:end]'  # Compute a sample of all the BV from the samples of b
EBV=mean(sampleEBV,2)                       # This is the posterior mean of the samples of BVs
EBVvar=cov(sampleEBV')                      # This is the prediction error variance
nothing

In [38]:
[EBV diag(EBVvar)]

1000x2 Array{Float64,2}:
 -0.683124    0.0242723 
 -0.857766    0.0234043 
 -0.150404    0.0150208 
 -2.03954     0.0223247 
 -1.47127     0.0249715 
 -0.101488    0.0156318 
  1.21938     0.010067  
 -0.00528541  0.00641935
 -0.407883    0.0128111 
 -0.710594    0.0126925 
 -0.150404    0.0150208 
  0.568271    0.0020361 
 -0.372332    0.0235937 
  ⋮                     
 -0.803565    0.0126986 
 -0.852481    0.0125045 
  0.201224    0.0144273 
 -0.586922    0.0110041 
 -0.81693     0.0273357 
 -0.150404    0.0150208 
  0.28406     0.0204651 
 -0.413168    0.0275046 
  0.34311     0.0194437 
 -1.33952     0.0366765 
  0.197992    0.0282716 
 -1.50887     0.0150365 